In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('/content/modified_traffic_data.csv')

In [3]:
features = data[['truck', 'car', 'bus', 'van']].values  # 4 input features
labels = data['total'].values  # 1 target label

# Convert to PyTorch tensors
features_tensor = torch.tensor(features, dtype=torch.float32)
labels_tensor = torch.tensor(labels, dtype=torch.float32).unsqueeze(1)

In [4]:
# Create DataLoader for training
dataset = TensorDataset(features_tensor, labels_tensor)
train_loader = DataLoader(dataset, batch_size=16, shuffle=True)

In [5]:
# Define the BiLSTM model
class BiLSTM_Model(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(BiLSTM_Model, self).__init__()
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, bidirectional=True, batch_first=True)
        self.fc = nn.Linear(hidden_size * 2, 1)  # *2 for bidirectional

    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.fc(x[:, -1, :])
        return x

In [6]:
# Model configuration
input_size = 4  # Four input features: truck, car, bus, van
hidden_size = 64
num_layers = 2
model = BiLSTM_Model(input_size, hidden_size, num_layers)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [7]:
# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for batch_features, batch_labels in train_loader:

        batch_features = batch_features.unsqueeze(1)

        # Forward pass
        outputs = model(batch_features)
        loss = criterion(outputs, batch_labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')

Epoch [1/50], Loss: 10927.3165
Epoch [2/50], Loss: 25.2945
Epoch [3/50], Loss: 4.9691
Epoch [4/50], Loss: 2.4438
Epoch [5/50], Loss: 2.3892
Epoch [6/50], Loss: 1.6823
Epoch [7/50], Loss: 2.7495
Epoch [8/50], Loss: 1.0393
Epoch [9/50], Loss: 1.8780
Epoch [10/50], Loss: 2.2064
Epoch [11/50], Loss: 1.1950
Epoch [12/50], Loss: 1.5144
Epoch [13/50], Loss: 2.8715
Epoch [14/50], Loss: 1.1611
Epoch [15/50], Loss: 1.1147
Epoch [16/50], Loss: 1.4860
Epoch [17/50], Loss: 1.6162
Epoch [18/50], Loss: 1.8540
Epoch [19/50], Loss: 0.9964
Epoch [20/50], Loss: 1.9194
Epoch [21/50], Loss: 1.8647
Epoch [22/50], Loss: 1.8124
Epoch [23/50], Loss: 1.6010
Epoch [24/50], Loss: 0.7947
Epoch [25/50], Loss: 1.3802
Epoch [26/50], Loss: 1.0532
Epoch [27/50], Loss: 1.4285
Epoch [28/50], Loss: 1.4708
Epoch [29/50], Loss: 1.1388
Epoch [30/50], Loss: 1.1477
Epoch [31/50], Loss: 1.5829
Epoch [32/50], Loss: 0.9574
Epoch [33/50], Loss: 0.9510
Epoch [34/50], Loss: 1.0472
Epoch [35/50], Loss: 1.0734
Epoch [36/50], Loss: 1.0

In [8]:
# Save the retrained model to a .pth file
model_path = 'traffic_model_features.pth'
torch.save(model.state_dict(), model_path)
print(f"Model saved as '{model_path}'")

Model saved as 'traffic_model_features.pth'


In [9]:
# Load the saved model
model = BiLSTM_Model(input_size, hidden_size, num_layers)
model.load_state_dict(torch.load('traffic_model_features.pth'))
model.eval()

<ipython-input-9-7a55cd453e27>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('traffic_model_features.pth'))


BiLSTM_Model(
  (lstm): LSTM(4, 64, num_layers=2, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=128, out_features=1, bias=True)
)

In [10]:
import torch
import pandas as pd

# Example: Predicting traffic volumes on new data
new_data = pd.DataFrame({
    'truck': [9, 2],  # New truck count
    'car': [20, 5],   # New car count
    'bus': [3, 1],    # New bus count
    'van': [5, 1]     # New van count
})

# Convert the new data to tensor
new_features = new_data.values
new_features_tensor = torch.tensor(new_features, dtype=torch.float32)

# Make sure the model is in evaluation mode
model.eval()

# Perform inference (prediction) on the new data
with torch.no_grad():

    predictions = model(new_features_tensor.unsqueeze(1))
    print(f'Predictions: {predictions.squeeze().numpy()}')


Predictions: [27.537489  9.007353]


In [11]:
actual_values = [30, 25]
predictions = predictions.squeeze()


from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(actual_values, predictions)
r2 = r2_score(actual_values, predictions)

print(f'Mean Squared Error (MSE): {mse:.4f}')
print(f'R-squared: {r2:.4f}')

Mean Squared Error (MSE): 130.9144
R-squared: -19.9463
